# Train a neural network.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
tfd = tfp.distributions

Set seeds

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

Set parameters

In [ ]:
# file with training input and output data: format is x,y
input_file = 'training.csv'

# names of input/output columns
inputs = ['mu', 'angle', 'threshold']
outputs = ['low_speed', 'high_speed']

# these set the input/output dimensions of the network
input_size = len(inputs)
output_size = len(outputs)

Read data

In [ ]:
data = pd.read_csv(input_file)
data.columns = inputs + outputs

x = np.array(data[inputs])
y = np.array(data[outputs])
y = y[...,1]
output_size = 1

Normalizing preprocessing layer from training data

In [ ]:
normalizer = keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x)

Penalize overpredictions more than underpredictions

In [ ]:
def asymmetric_mse(y_true, y_pred):
    standard_mse = keras.losses.mse(y_true, y_pred)
    geq = keras.backend.any(keras.backend.greater(y_pred, y_true)) # true/false, are there overpredictions?
    geq_scale = keras.backend.switch(geq,5.0,1.0) # if there are overpredictions, scale up mse
    return geq_scale * standard_mse

Base model

In [ ]:
# model = keras.models.Sequential()
# model.add(keras.layers.Dense(input_size, activation='linear'))
# #model.add(keras.layers.Dense(20,activation='relu'))
# model.add(keras.layers.Dense(4,activation='relu'))
# model.add(keras.layers.Dense(output_size,activation='relu'))
# model.compile(loss='mse',optimizer='adam')

In [ ]:
negloglik = lambda y, rv_y: -rv_y.log_prob(y)

In [ ]:
# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
  ])

# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
  ])

In [ ]:
model = keras.models.Sequential()
#model.add(keras.layers.Dense(input_size, activation='linear'))
#model.add(keras.layers.Dense(20,activation='relu'))
model.add(tfp.layers.DenseVariational(2,posterior_mean_field, prior_trainable))
model.add(tfp.layers.DistributionLambda(
      lambda t: tfd.Normal(loc=t[..., :1],
                           scale=1e-3 + tf.math.softplus(0.01 * t[..., 1:]))))
#model.add(keras.layers.Dense(output_size,activation='relu'))
model.compile(loss=negloglik,optimizer='adam')

Compile model

In [ ]:
# input_shape = x.shape[1:]
# full_model_input = keras.Input(shape=input_shape)
# normalized_input = normalizer(full_model_input)
# full_model_output = model(normalized_input)
# full_model = keras.Model(full_model_input, full_model_output)
# full_model.compile(loss=negloglik,optimizer='adam')
full_model = model

In [ ]:
full_model = tf.keras.Sequential([
  tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_trainable, kl_weight=1/x.shape[0]),
  tfp.layers.DistributionLambda(
      lambda t: tfd.Normal(loc=t[..., :1],
                           scale=1e-3 + tf.math.softplus(0.01 * t[...,1:]))),
])
full_model.compile(loss=negloglik,optimizer='adam',metrics=['mse'])

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, shuffle= True)

Train model

In [ ]:
model_output = full_model.fit(x_train,y_train,epochs=3000,batch_size=10,verbose=0,validation_data=(x_val,y_val)) # check validation

View training

In [ ]:
cut = 0
plt.yscale('log')
plt.title('loss')
plt.plot(model_output.history['loss'][cut:], label='train')
plt.plot(model_output.history['val_loss'][cut:], label='validation')
plt.legend()
plt.figure()
plt.yscale('log')
plt.title('mse')
plt.plot(model_output.history['mse'][cut:], label='train')
plt.plot(model_output.history['val_mse'][cut:], label='validation')
plt.legend()
plt.figure()

View output

In [ ]:
temp = full_model.predict(1000*[[0.009,90,4]])
plt.hist(temp)
print(temp.std())
print(temp.mean())

In [ ]:
# plot speed vs angle given mu, threshold
mu = 1 # set mu
thresh = 4 # set threshold

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# this is not generic enough...
pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
pred = full_model.predict(pred_x)

#plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
plt.plot([i[1] for i in plot_x], [y[i] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh])
plt.plot([i[1] for i in pred_x], pred)

In [ ]:
#plt.figure(figsize=[6, 1.5])  # inches
plt.plot(x, y, 'b.', label='observed');
x_tst = x

yhats = [model(x_tst) for _ in range(100)]
avgm = np.zeros_like(x_tst[..., 0])
for i, yhat in enumerate(yhats):
  m = np.squeeze(yhat.mean())
  s = np.squeeze(yhat.stddev())
  if i < 15:
    plt.plot(x_tst, m, 'r', label='ensemble means' if i == 0 else None, linewidth=1.)
    plt.plot(x_tst, m + 2 * s, 'g', linewidth=0.5, label='ensemble means + 2 ensemble stdev' if i == 0 else None);
    plt.plot(x_tst, m - 2 * s, 'g', linewidth=0.5, label='ensemble means - 2 ensemble stdev' if i == 0 else None);
  avgm += m
plt.plot(x_tst, avgm/len(yhats), 'r', label='overall mean', linewidth=4)

#plt.ylim(-0.,17);
#plt.yticks(np.linspace(0, 15, 4)[1:]);
#plt.xticks(np.linspace(*x_range, num=9));

ax=plt.gca();
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.spines['left'].set_position(('data', 0))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['left'].set_smart_bounds(True)
#ax.spines['bottom'].set_smart_bounds(True)
plt.legend(loc='center left', fancybox=True, framealpha=0., bbox_to_anchor=(1.05, 0.5))


In [ ]:
yhats = [full_model(x) for _ in range(100)]

In [ ]:
avgm = np.zeros_like(x_tst[..., 0])
for i, yhat in enumerate(yhats):
  m = np.squeeze(yhat.mean())
  s = np.squeeze(yhat.stddev())
  avgm += m
plt.plot(x_tst, avgm/len(yhats), 'r', label='overall mean', linewidth=4)

Save model

In [ ]:
# class CustomVariational(tfp.layers.DenseVariational):
#   def get_config(self):
#         config = super().get_config().copy()
#         config.update({
#             'units': self.units,
#             'make_posterior_fn': self._make_posterior_fn,
#             'make_prior_fn': self._make_prior_fn
#         })
#         return config

In [ ]:
# full_model.save("2_speed_network_bayesian.h5")
keras.models.save_model(full_model,"2_speed_network_bayesian.h5")

Find largest divergence between prediction and training data

In [ ]:
divergence = (full_model.predict(x) - y)
max_divergence = max([i[1] for i in divergence])
print(max_divergence)
#print(max_divergence, x[np.where(divergence == max_divergence)[0]])

Make a lot of plots

In [ ]:
# only plot the high prediction...
mus = data['mu'].unique()
angles = data['angle'].unique()
thresholds = data['threshold'].unique()
a=0
# plot speed vs angle given mu, threshold
for mu in mus:
    for threshold in thresholds:
        # bug: mu = 0.009 is read as 0.0090..01
        #plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
        plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == threshold] # x, y
        pred_x = [[mu,angle,threshold] for angle in np.linspace(angles.min(),angles.max(),angles.max())]
        pred = full_model.predict(pred_x)
        #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
        fig = plt.figure()
        plt.title('mu: %.3f, threshold: %.2f' % (mu, threshold))
        plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == threshold],
                label = 'training')
        plt.plot([i[1] for i in pred_x], [i[1] for i in pred], label = 'predicted')
        plt.legend()
        plt.savefig('plots/mu-%.3f_threshold-%.2f.png' % (mu,threshold))
        plt.close()